<h3>Report</h3>

In [1]:
import sys
sys.path.append('..')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pickle

<h3>Loading Models</h3>

In [3]:
from core.network import *
from core.trainer import *

In [4]:
device = torch.device("cuda:0")
dir_load = '../models/'

In [5]:
net = LSTM(1, 98, 9, num_layers=3, device=device).to(device)
model = LSTMTrainer(net, device=device)

In [6]:
if dir_load is not None:
    model.load(dir_load + 'lstm1.pth')
    #model.load(dir_load + 'cls.pth')

<h3>Dataloading for Eval</h3>

In [7]:
ligands = ["TNF", "R84", "PIC", "P3K", "FLA", "CpG", "FSL", "LPS", "UST"]
polarization = ["", "ib", "ig", "i0", "i3", "i4"]
replicas, size = 2, 1288

In [8]:
from core.getdata import *
from core.dataset import * 
from core.dataset2 import *

In [9]:
data = Dataset(ligands, polarization, replicas, size)
data2 = DatasetNaive(ligands, replicas, size)

In [10]:
from torch.utils.data import Dataset
from torchvision import datasets

In [11]:
print(len(data), 0.25 * len(data)) # we'll test our data on 25% of the dataset

69552 17388.0


In [32]:
_, datatest = torch.utils.data.random_split(data, [69552 - 17388 - 24, 17388 + 24])
testloader = torch.utils.data.DataLoader(datatest, batch_size=32, shuffle=True)

<h3>Evaluation Loop</h3>

In [63]:
polarization_true, polarization_pred = np.empty([32,]).T, np.empty([32,]).T

In [64]:
for x, y in testloader:
    x, y = x.to(device), y.to(device)
    y_true = y.detach().cpu().numpy()
    polarization_true = np.hstack([polarization_true, y_true.T])
    y_pred = model.network(x)
    y_pred = F.softmax(y_pred, dim=1)
    y_pred = y_pred.detach().cpu().numpy()
    y_pred = np.argmax(y_pred, axis=1)
    #import pdb; pdb.set_trace()
    polarization_pred = np.hstack([polarization_pred, y_pred.T])
    #y_true = (32, ); y_pred = (32, )

In [65]:
polarization_true, polarization_pred = polarization_true[32:], polarization_pred[32:]
print(polarization_true.shape, polarization_pred.shape)

(17412,) (17412,)


In [74]:
comparison = np.vstack([polarization_true, polarization_pred]).T

In [77]:
df = pd.DataFrame(comparison)

In [78]:
df

,0,1
0,7.0,7.0
1,4.0,4.0
2,0.0,0.0
3,1.0,1.0
4,5.0,6.0
...,...,...
17407,5.0,5.0
17408,3.0,3.0
17409,0.0,0.0
17410,4.0,4.0


In [83]:
print(f' Accuracy: {sum(df[0] == df[1])/17412}')

 Accuracy: 0.9147714220078107
